Считываем данные из файлов и сохраняем их в pandas dataframes...

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("linear_train.txt", names=['word', 'is_surname'])
test = pd.read_table("linear_test.txt", header=None)
answer = pd.read_csv("linear_ans_example.txt")

In [3]:
test.head()

,0
0,Аалто
1,ААР
2,Аара
3,Ааре
4,Аарон


In [4]:
train.head()

,word,is_surname
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


In [5]:
answer.head()

,Id,Answer
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


Разбиваем полученную таблицу train на feature и target, получаем признаковое описание для каждого слова...

In [12]:
words = train['word'].tolist()
target = train['is_surname'].tolist()
test_words =  test[0].tolist()

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(lowercase=False, analyzer='char', ngram_range=(3, 7))
count_vectorizer.fit(words)
feature_matrix = count_vectorizer.transform(words)
test_feature_matrix = count_vectorizer.transform(test_words)

Обучаем модель...

In [13]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss='log', penalty='l2', power_t=0.5,
                      learning_rate='invscaling', eta0=3.0, alpha=0.00022)

Смотрим на roc-auc в кроссвалидации. Подбираем параметры CountVectorizer'а и модели, максимизируя cross_val_score...

In [14]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model, feature_matrix, target, scoring =  'roc_auc', cv = 3).mean()
print(score)

0.819721250126


Использовались SGDClassifier, LogisticReression, SVM, LinearRegression. Наилучший результат на кросвалидации показал первый.

Предсказываем на тестовой выборке, сохраняем, отправляем...

In [45]:
model.fit(feature_matrix, target)
res = model.predict_proba(test_feature_matrix)

In [46]:
res=[a[1] for a in res]
answer['Answer']=res

In [47]:
answer.to_csv("answer.tsv", sep=',', index=False)